Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [159]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice, uniform
from copy import deepcopy
import numpy as np
from tqdm import trange

In [160]:
State = namedtuple("State", ["x", "o"])
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]
winning_formations = [
    [2, 7, 6], [9, 5, 1], [4, 3, 8],  # Rows
    [2, 9, 4], [7, 5, 3], [6, 1, 8],  # Columns
    [2, 5, 8], [6, 5, 4]              # Diagonals
]

In [161]:
def win(elements):
    """Checks if elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))


def state_value(pos: State, action):
    """Evaluate state: +1 first player wins"""
    state_x = deepcopy(pos.x)
    if action is not None:
        state_x.add(action)
    state_o = deepcopy(pos.o)
    if action is not None:
        state_o.add(action)
    if win(state_x):
        return 1
    elif win(state_o):
        return -1
    elif action is not None and intermediate_reward_condition(pos, action):
        return 0.5
    else:
        return 0


def intermediate_reward_condition(state, action):
    # Check if a player has made a move forming a potential winning formation
    # For example, checking if two cells in a row/column/diagonal are taken by the same player
    # and the third cell is available

    for formation in winning_formations:
        # check for two cells taken by the player and the third cell available
        new_state = deepcopy(state)
        new_state.x.add(action)
        x_old_count = sum(1 for cell in formation if cell in state.x)
        x_count = sum(1 for cell in formation if cell in new_state.x)
        o_count = sum(1 for cell in formation if cell in state.o)
        if x_count == 2 and o_count == 0 and x_old_count == 1:
            empty_cell = [
                cell
                for cell in formation
                if cell not in new_state.x and cell not in state.o
            ]
            if empty_cell:
                return True

    return False


def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print("X", end="")
            elif MAGIC[i] in pos.o:
                print("O", end="")
            else:
                print(".", end="")
        print()
    print()

In [162]:
def epsilon_greedy_policy(Qtable, state, epsilon, available):
    state_key = state_to_key(state)
    random_int = uniform(0, 1)
    if random_int > epsilon:
        actions = sorted(Qtable[state_key], key=lambda k: Qtable[state_key][k], reverse=True)
        if len(actions) == 0:
            action = choice(list(available))
        for a in actions:
            if a in available:
                action = a
                break
    else:
        action = choice(list(available))
    return action


def state_to_key(state):
    return (frozenset(state.x), frozenset(state.o))


def ply(state, epsilon, learning_rate, gamma, Qtable, available):
    if not available:
        action = None
        hashable_state = state_to_key(state)
        current_Q = Qtable[hashable_state][action]
        reward = state_value(state, action)
        Qtable[hashable_state][action] = (1 - learning_rate) * current_Q + learning_rate * reward
        return
    
    action = epsilon_greedy_policy(Qtable, state, epsilon, available)

    reward = state_value(state, action)

    new_state = deepcopy(state)
    new_state.x.add(action)

    hashable_new_state = state_to_key(new_state)
    hashable_state = state_to_key(state)

    current_Q = Qtable[hashable_state][action]
    
    if hashable_new_state in Qtable and Qtable[hashable_new_state]:
        next_max_Q = (
            max(Qtable[hashable_new_state].values())
        )
    else: 
        next_max_Q = 0
    

    # Q-value update via Bellman's equation
    Qtable[hashable_state][action] = learning_rate * (
        reward - gamma * next_max_Q - current_Q
    )

    return action

In [163]:
def train(n_training_episodes, learning_rate, min_epsilon, max_epsilon, decay_rate, gamma, Qtable):
    for episode in range(n_training_episodes):
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(
            -decay_rate * episode
        )
        # q-learn player is first
        state = State(set(), set())

        available = set(range(1, 9 + 1))
        while available:
            x = ply(state, epsilon, learning_rate, gamma, Qtable, available)
            if x is not None:
                state.x.add(x)
                available.remove(x)
            if win(state.x) or not available:
                break

            o = choice(list(available))
            state.o.add(o)
            available.remove(o)

        # q-learn player is second
        state = State(set(), set())

        available = set(range(1, 9 + 1))
        while available:
            o = choice(list(available))
            state.o.add(o)
            available.remove(o)

            x = ply(state, epsilon, learning_rate, gamma, Qtable, available)
            if x is not None:
                state.x.add(x)
                available.remove(x)
            if win(state.x) or not available:
                break

    return Qtable

In [164]:
def q_agent_turn(Qtable, epsilon, state, available):    
    x = epsilon_greedy_policy(Qtable, state, epsilon, available)
    state.x.add(x)
    available.remove(x)
    if win(state.x):
        return "wins"
    if not available:
        return "draws"
    return None
        
def random_agent_turn(state, available):
    o = choice(list(available))
    state.o.add(o)
    available.remove(o)
    if win(state.o):
        return "losses"
    if not available:
        return "draws"
    return None

In [165]:
def test(n_training_episodes, Qtable, min_epsilon, max_epsilon, decay_rate, q_agent_first):
    # evaluation function of q-player against an opponent who plays randomly for n_training_episodes
    # with q-learn player playing first and n_training_episodes playing second
    
    results = dict()
    results["wins"] = 0
    results["losses"] = 0
    results["draws"] = 0

    for episode in trange(n_training_episodes):
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(
            -decay_rate * episode
        )
        state = State(set(), set())
        available = set(range(1, 9 + 1))
        while available:
            if q_agent_first:
                res = q_agent_turn(Qtable, epsilon, state, available)
                if res is not None: 
                    results[res] += 1
                    break
                res = random_agent_turn(state, available)
                if res is not None: 
                    results[res] += 1
                    break
            else:
                res = random_agent_turn(state, available)
                if res is not None: 
                    results[res] += 1
                    break
                res = q_agent_turn(Qtable, epsilon, state, available)
                if res is not None: 
                    results[res] += 1
                    break

    return results

In [166]:
Qtable_ttt = defaultdict(lambda: defaultdict(lambda: 0.0))

n_training_episodes = 500_000
learning_rate = 0.3

n_eval_episodes = 1000

gamma = 0.95

max_epsilon = 0.5
min_epsilon = 0.2
decay_rate = 10e-10

Qtable_ttt = train(n_training_episodes, learning_rate, min_epsilon, max_epsilon, gamma, decay_rate, Qtable_ttt)

max_epsilon = 0.2
min_epsilon = 0.0
decay_rate = 10e-10

results = test(n_eval_episodes, Qtable_ttt, min_epsilon, max_epsilon, decay_rate, True)
print(f"Q-Learn agent wins {results['wins'] * 100 / n_eval_episodes}%, loses {results['losses'] * 100 / n_eval_episodes}% and draws {results['draws'] * 100 / n_eval_episodes}% of the time when playing first")

results = test(n_eval_episodes, Qtable_ttt, min_epsilon, max_epsilon, decay_rate, False)
print(f"Q-Learn agent wins {results['wins'] * 100 / n_eval_episodes}%, loses {results['losses'] * 100 / n_eval_episodes}% and draws {results['draws'] * 100 / n_eval_episodes}% of the time when playing second")

100%|██████████| 1000/1000 [00:00<00:00, 96747.72it/s]


Q-Learn agent wins 90.8%, loses 6.9% and draws 2.3% of the time when playing first


100%|██████████| 1000/1000 [00:00<00:00, 82354.29it/s]

Q-Learn agent wins 65.4%, loses 32.7% and draws 1.9% of the time when playing second
